In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

Warning: Subroutine cached_single_arg redefined at graphql.pm line 220.


In [2]:
my $practice_input1 = qq/
AAAA
BBCD
BBCC
EEEC
/;
my $practice_input2 = qq/
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
/;
my $practice_input3 = qq/
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
/;


RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE


In [65]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub stringp { my ($p) = $_[0] // $_; return $p->[0].','.$p->[1] }
sub uniqp { my %h; @h{map $_->[0].','.$_->[1], @_} = (); map [ split ',' ], keys %h }
sub confirm { my ($a, $b) = @_; die "expiremental '$a' doesn't match expected $b" unless $a eq $b; say "confirmed: $a"; return $a }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}
# $graphql::graphql_methods{is_connected} = sub {
#         my ($self, $params) = @_;
#         my @row = @{$self->{arr}[$self->{coord}[0]]};
#         return abs($self->{value}) ;
#     };
$graphql::graphql_methods{spf_search} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        my @chains = ({ value => $self->{value}, chain => [ $self->{coord} ] });
        my @shape = shape($self->{arr});
        my $steps = 0;
        while (@chains && $steps < 9) {
            @chains =
                map { my ($chain, $next, $next_val) = @$_; { value => $next_val, chain => [ @{$chain->{chain}}, $next ] } }
                grep { my ($chain, $next, $next_val) = @$_; $next_val == ($chain->{value} + 1) }
                map { my $chain = $_; map [ $chain, $_, $arr->[$_->[0]][$_->[1]] ], adjacent_points($_->{chain}[-1], @shape) }
                @chains;
            $steps++;
        }
        return { chains => [ @chains ] };
    };

CODE(0x5558a16a3728)

Warning: Subroutine subp redefined at reply input line 1.

Subroutine addp redefined at reply input line 2.

Subroutine stringp redefined at reply input line 3.

Subroutine uniqp redefined at reply input line 4.

Subroutine in_bounds redefined at reply input line 6.

Subroutine adjacent_points redefined at reply input line 10.


In [66]:

$graphql::graphql_methods{flood_group} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        my @shape = shape($self->{arr});

        state %flood_group_cache;

        return $flood_group_cache{$arr}{stringp($self->{coord})} if exists $flood_group_cache{$arr}{stringp($self->{coord})};
        
        my $key = $self->{value};
        my %group;
        my @next_points = ($self->{coord});
        my $rep = 0;
        while (@next_points) {
            die if $rep++ > 50;
            @group{map stringp, @next_points} = ();
            @next_points =
                uniqp
                grep { not exists $group{stringp $_} }
                grep $arr->[$_->[0]][$_->[1]] eq $key,
                map adjacent_points($_, @shape),
                @next_points;
        }
        my $ret = [ map [split ','], keys %group ];
        foreach (@$ret) {
            $flood_group_cache{$arr}{stringp $_} = $ret;
        }
        return $ret;
    };
$graphql::graphql_methods{group_by} = sub {
        my ($self, $params) = @_;
        my $key = $params->{key};
        my $arr = $self->{arr};

        state $cached //= {};

        $cached->{"$arr/$key"} = {
            group_by selector($key),
            flatten_nd
            $self->{arr}
        } unless exists $cached->{"$arr/$key"};
        return $cached->{"$arr/$key"};
    };
my @pos_table = 'a' .. 'z';
$graphql::graphql_methods{is_different_neighbor} = sub {
        my ($self, $params) = @_;
        my $new_coord = [ map { $params->{"abs_$pos_table[$_]"} // ($self->{coord}[$_] + ($params->{"d$pos_table[$_]"} // 0)) } 0 .. $#{$self->{coord}} ];
        my $other = graphql::get_value($self->{arr}, $new_coord);
        if (defined $other) {
            return $other ne $self->{value};
        }
        return 1;
    };
$graphql::graphql_methods{perimeter} = sub {
        my ($self, $params) = @_;
        return $self->{is_different_neighbor}($self, { da => -1 })
            + $self->{is_different_neighbor}($self, { da => 1 })
            + $self->{is_different_neighbor}($self, { db => -1 })
            + $self->{is_different_neighbor}($self, { db => 1 });
    };
sub process_day12_part1 {
    my ($input) = @_;
    my $table =
        map_nd_indexed graphql_query('{ group_by(key:"flood_group") }'),
        map_nd_indexed graphql_query(qq/{
            flood_group
            value
            perimeter
        }/),
        parse_2d_map_array($input);
    my $groups = $table->[0][0]{group_by};
    my @calculated_areas = map { {
        area => $groups->{$_}[0]{flood_group},
        value => $groups->{$_}[0]{value},
        perimeter => sum map $_->{perimeter}, @{$groups->{$_}}
    } } keys %$groups;
    
    sum map @{$_->{area}} * $_->{perimeter}, @calculated_areas
}
timer sub {
    say Dumper process_day12_part1($practice_input3);
    confirm process_day12_part1($practice_input1), 140;
    confirm process_day12_part1($practice_input2), 772;
    confirm process_day12_part1($practice_input3), 1930;
}

$VAR1 = 1930;

confirmed: 140
confirmed: 772
confirmed: 1930
[time] 0 seconds


1

Warning: Subroutine process_day12_part1 redefined at reply input line 62.


In [67]:
timer sub {
    my $input = get_challenge('2024/day/12/input');
    my $res = process_day12_part1($input);
    confirm $res, 1489582;
}

confirmed: 1489582
[time] 1 seconds


1

In [64]:
say post_answer('2024/day/12/answer', 1, $res); # 1489582

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 12 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [86]:

my @pos_table = 'a' .. 'z';
$graphql::graphql_methods{is_different_neighbor_contiguous} = sub {
        my ($self, $params) = @_;
        my $delta = [ map { $params->{"abs_$pos_table[$_]"} // ($params->{"d$pos_table[$_]"} // 0) } 0 .. $#{$self->{coord}} ];
        my $perpendicular = [ $delta->[1], -$delta->[0] ];
        my $wall_coord = addp($delta, $self->{coord});
        my $perp_coord = addp($perpendicular, $self->{coord});
        my $perp_wall_coord = addp($delta, addp($perpendicular, $self->{coord}));
        my $wall = graphql::get_value($self->{arr}, $wall_coord);
        my $perp = graphql::get_value($self->{arr}, $perp_coord);
        my $perp_wall = graphql::get_value($self->{arr}, $perp_wall_coord);
        if (defined $wall and $wall eq $self->{value}) {
            return 0;
        # } elsif (not defined $wall or $wall ne $self->{value}) {
        } else {
            return 1 unless (defined $perp and $perp eq $self->{value});
            if (defined $perp_wall and $perp_wall eq $self->{value}) {
                return 1;
            } else {
                return 0;
            }
        }
    };
$graphql::graphql_methods{perimeter_contiguous} = sub {
        my ($self, $params) = @_;
        return $self->{is_different_neighbor_contiguous}($self, { da => -1 })
            + $self->{is_different_neighbor_contiguous}($self, { da => 1 })
            + $self->{is_different_neighbor_contiguous}($self, { db => -1 })
            + $self->{is_different_neighbor_contiguous}($self, { db => 1 });
    };

sub process_day12_part2 {
    my ($input) = @_;
    my $table =
        map_nd_indexed graphql_query('{ group_by(key:"flood_group") }'),
        map_nd_indexed graphql_query(qq/{
            flood_group
            value
            perimeter:perimeter_contiguous
        }/),
            # up:is_different_neighbor_contiguous(da:-1)
            # down:is_different_neighbor_contiguous(da:1)
            # left:is_different_neighbor_contiguous(db:-1)
            # right:is_different_neighbor_contiguous(db:1)
        parse_2d_map_array($input);
    my $groups = $table->[0][0]{group_by};
    my @calculated_areas = map { {
        area => $groups->{$_}[0]{flood_group},
        value => $groups->{$_}[0]{value},
        perimeter => sum map $_->{perimeter}, @{$groups->{$_}}
    } } keys %$groups;
    
    sum map @{$_->{area}} * $_->{perimeter}, @calculated_areas
}
timer sub {
    say Dumper process_day12_part2(qq/
AAAA
BBCD
BBCC
EEEC
/);
    confirm process_day12_part2($practice_input1), 80;
    confirm process_day12_part2($practice_input2), 436;
    confirm process_day12_part2($practice_input3), 1206;
    confirm process_day12_part2(qq/
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA/), 368;
}

$VAR1 = 80;

confirmed: 80
confirmed: 436
confirmed: 1206
confirmed: 368
[time] 0 seconds


1

Warning: Subroutine process_day12_part2 redefined at reply input line 33.


In [93]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/12/input');
    $res = process_day12_part2($input);
    say Dumper $res;
    confirm $res, 914966;
}

$VAR1 = 914966;

confirmed: 914966
[time] 2 seconds


1

In [91]:
$res

914966

In [92]:
say post_answer('2024/day/12/answer', 2, $res); # 914966

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 12 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1